In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [ ]:
import makeAmoebasX
import amoeba2Dxx
import amoebaSegments2x
import draw_matplotlib
import render

In [ ]:
# Defaults are given in amoeba_struct (like in Matlab). Image with target:
amoeba_struct = makeAmoebasX.amoeba_struct 

amoeba_struct.num_targets = 1
amoeba_struct.num_distractors   = 3

# Set these as desired:
which_target=101
which_clutter=100
[s,xs,ys]=amoeba2Dxx.amoeba2Dxx( amoeba_struct, which_target, which_clutter )
# Render as white (255) on black background (0):

buf = np.zeros((amoeba_struct.image_rect_size, amoeba_struct.image_rect_size), dtype=np.uint8)
img=render.render_amoeba(buf, xs,ys)

#Draw:
plt.imshow(img, cmap='bone')
plt.figure();

# Render as black on white:
buf = 255-np.zeros((amoeba_struct.image_rect_size, amoeba_struct.image_rect_size), dtype=np.uint8)
img=render.render_amoeba(buf, xs,ys,0)
plt.imshow(img, cmap='bone')



In [ ]:
# Non-target image. Has 4 distractors and no target.
amoeba_struct = makeAmoebasX.amoeba_struct 

amoeba_struct.num_targets = 0
amoeba_struct.num_distractors   = 4

which_clutter=100
[s,xs,ys]=amoeba2Dxx.amoeba2Dxx( amoeba_struct, -1, which_clutter )

# Black on white:
buf = np.zeros((amoeba_struct.image_rect_size, amoeba_struct.image_rect_size), dtype=np.uint8)
img=render.render_amoeba(buf, xs,ys)
plt.imshow(255-img, cmap='bone')